In [1]:
from ConllDataLoader import conll_dataset
data = conll_dataset('../data/new_train_post.conll')

texts, tags = data["tokens"], data["ner_tags"]

c:\Softwares\Research\envs\sci_rec\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The conll file is probably using spaces to separate tokens and ner tags. Trying to parse using spaces.


In [2]:
data.id2tag.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])

In [3]:
#Start from here!
from ConllDataLoader import conll_dataset

train_data_split = conll_dataset('../data/train.conll')
train_texts, train_tags = train_data_split["tokens"], train_data_split["ner_tags"]

val_data_split = conll_dataset('../data/test.conll')
val_texts, val_tags = val_data_split["tokens"], val_data_split["ner_tags"]

The conll file is probably using spaces to separate tokens and ner tags. Trying to parse using spaces.
The conll file is probably using spaces to separate tokens and ner tags. Trying to parse using spaces.


In [4]:
len(train_texts), len(val_texts)

(1566, 1044)

In [5]:
#unique_tags = set(tag for doc in tags for tag in doc)
tag2id = data.tag2id#{tag: id for id, tag in enumerate(unique_tags)}
id2tag = data.id2tag#{id: tag for tag, id in tag2id.items()}
unique_tags = set(tag2id.keys())
unique_tags

{'B-DatasetName',
 'B-HyperparameterName',
 'B-HyperparameterValue',
 'B-MethodName',
 'B-MetricName',
 'B-MetricValue',
 'B-TaskName',
 'I-DatasetName',
 'I-HyperparameterName',
 'I-HyperparameterValue',
 'I-MethodName',
 'I-MetricName',
 'I-MetricValue',
 'I-TaskName',
 'O'}

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
# train_encodings = tokenizer(train_texts, is_split_into_words=True, truncation=True)
# val_encodings = tokenizer(val_texts, is_split_into_words=True, truncation=True)
# tokenizer.is_fast

In [7]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [8]:
def tokenize_and_align_labels(tokens, ner_tags, tokenizer):
    tokenized_inputs = tokenizer(
        tokens, truncation=True, is_split_into_words=True,
    )
    all_labels = ner_tags
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    #tokenized_inputs["labels"] = new_labels
    return tokenized_inputs, new_labels

In [9]:
import torch

class TokenizedDataset(torch.utils.data.Dataset):
    def __init__(self, tokenized_inputs, new_labels):
        self.encodings = tokenized_inputs
        self.labels = new_labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

In [10]:
train_encodings, train_encoded_labels = tokenize_and_align_labels(train_texts, train_tags,tokenizer)
train_dataset = TokenizedDataset(train_encodings, train_encoded_labels)
val_encodings, val_encoded_labels = tokenize_and_align_labels(val_texts, val_tags,tokenizer)
val_dataset = TokenizedDataset(val_encodings, val_encoded_labels)

In [11]:
label_names = list(id2tag.values())
label_names

['O',
 'B-DatasetName',
 'I-DatasetName',
 'B-HyperparameterName',
 'I-HyperparameterName',
 'B-HyperparameterValue',
 'I-HyperparameterValue',
 'B-MethodName',
 'I-MethodName',
 'B-MetricName',
 'I-MetricName',
 'B-MetricValue',
 'I-MetricValue',
 'B-TaskName',
 'I-TaskName']

In [12]:
import numpy as np
import evaluate

metric = evaluate.load("seqeval")


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [13]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [14]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained('bert-base-cased', id2label = id2tag, label2id=tag2id,num_labels=len(unique_tags))
len(unique_tags)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

15

In [15]:
from transformers import AutoModelForTokenClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='../models/bert_base_cased',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=2,  # batch size per device during training
    per_device_eval_batch_size=4,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='../models/logs/bert_base_cased',            # directory for storing logs                               logging_steps=10,
    save_strategy='epoch',
    evaluation_strategy='epoch'
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,
    data_collator=data_collator,
    compute_metrics=compute_metrics,     # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()
tokenizer.save_pretrained("../models/bert-base-nlp")
model.save_pretrained("../models/bert-base-nlp")

c:\Softwares\Research\envs\sci_rec\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1566
  Num Epochs = 2
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 1566
 32%|███▏      | 502/1566 [00:47<01:34, 11.21it/s]

{'loss': 0.7011, 'learning_rate': 5e-05, 'epoch': 0.64}


 50%|████▉     | 782/1566 [01:12<01:09, 11.24it/s]***** Running Evaluation *****
  Num examples = 1044
  Batch size = 4
                                                  
 50%|█████     | 783/1566 [01:19<01:09, 11.24it/s]Saving model checkpoint to ../models/bert_base_cased\checkpoint-783
Configuration saved in ../models/bert_base_cased\checkpoint-783\config.json


{'eval_loss': 0.25050726532936096, 'eval_precision': 0.3603715170278638, 'eval_recall': 0.44769230769230767, 'eval_f1': 0.39931389365351627, 'eval_accuracy': 0.9255160672483507, 'eval_runtime': 7.2308, 'eval_samples_per_second': 144.382, 'eval_steps_per_second': 36.095, 'epoch': 1.0}


Model weights saved in ../models/bert_base_cased\checkpoint-783\pytorch_model.bin
 64%|██████▍   | 1002/1566 [01:47<00:49, 11.48it/s]

...

100%|██████████| 1566/1566 [02:38<00:00, 11.38it/s]***** Running Evaluation *****
  Num examples = 1044
  Batch size = 4
                                                   
100%|██████████| 1566/1566 [02:45<00:00, 11.38it/s]Saving model checkpoint to ../models/bert_base_cased\checkpoint-1566
Configuration saved in ../models/bert_base_cased\checkpoint-1566\config.json


{'eval_loss': 0.18736249208450317, 'eval_precision': 0.5895522388059702, 'eval_recall': 0.6684615384615384, 'eval_f1': 0.6265320836337419, 'eval_accuracy': 0.9480132550998692, 'eval_runtime': 7.1707, 'eval_samples_per_second': 145.592, 'eval_steps_per_second': 36.398, 'epoch': 2.0}


Model weights saved in ../models/bert_base_cased\checkpoint-1566\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 1566/1566 [02:49<00:00,  9.27it/s]
tokenizer config file saved in ../models/bert-base-nlp\tokenizer_config.json
Special tokens file saved in ../models/bert-base-nlp\special_tokens_map.json
Configuration saved in ../models/bert-base-nlp\config.json


{'train_runtime': 169.0159, 'train_samples_per_second': 18.531, 'train_steps_per_second': 9.265, 'train_loss': 0.3614274575488016, 'epoch': 2.0}


Model weights saved in ../models/bert-base-nlp\pytorch_model.bin


In [16]:
from transformers import AutoModelForTokenClassification
#tokenizer.save_pretrained()
saved_ckpt = "../models/bert-base-nlp"
saved_model = AutoModelForTokenClassification.from_pretrained(saved_ckpt)

loading configuration file ../models/bert-base-nlp\config.json
Model config BertConfig {
  "_name_or_path": "../models/bert-base-nlp",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-DatasetName",
    "2": "I-DatasetName",
    "3": "B-HyperparameterName",
    "4": "I-HyperparameterName",
    "5": "B-HyperparameterValue",
    "6": "I-HyperparameterValue",
    "7": "B-MethodName",
    "8": "I-MethodName",
    "9": "B-MetricName",
    "10": "I-MetricName",
    "11": "B-MetricValue",
    "12": "I-MetricValue",
    "13": "B-TaskName",
    "14": "I-TaskName"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-DatasetName": 1,
    "B-HyperparameterName": 3,
    "B-HyperparameterValue": 5,
    "B-MethodName": 7,
    "B-MetricName":

In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(saved_ckpt)

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json


In [18]:
def test_output_model(test_data_file, test_output_file, tokenizer, saved_model):
    with open(test_output_file, 'wt', encoding='utf-8') as w:
        with open(test_data_file, 'rt', encoding='utf-8') as f:
            for paragraph in f.read().split("\n"):
                if paragraph == "":
                    break
                tokens = paragraph.split(' ')
                test_encodings = tokenizer(tokens, is_split_into_words=True, truncation=True, return_tensors='pt')#,return_offsets_mapping=True)
                #offset_mapp = test_encodings.offset_mapping
                test_encodings.pop("token_type_ids")
                #test_encodings.pop("offset_mapping")
                outputs = saved_model(**test_encodings)

                predictions = outputs.logits.argmax(dim=-1).squeeze()
                    
                prev_word_id = None
                output_ner_tags = []
                for i, word_id in enumerate(test_encodings.word_ids()):
                    if word_id == None:
                        #special token here
                        pass
                    else:
                        if prev_word_id != word_id:
                            curr_tag = saved_model.config.id2label[int(predictions[i])]
                            if curr_tag.startswith("I"):
                                if word_id>=1 and output_ner_tags[word_id-1]=="O":
                                    output_ner_tags[word_id-1] = "B-"+curr_tag.split("-")[-1]
                                elif word_id==0:
                                    curr_tag = "B-"+curr_tag.split("-")[-1]
                            output_ner_tags.append(curr_tag)
                            prev_word_id = word_id
                for i in range(len(tokens)):
                    if i>=len(output_ner_tags):
                        w.write(tokens[i]+"\t"+"O"+"\n")
                    else:
                        w.write(tokens[i]+"\t"+output_ner_tags[i]+"\n")
                w.write("\n")
        


In [19]:
test_output_model(test_data_file="../data/test.txt", test_output_file= "../data/test-bert-base-cased.conll", tokenizer=tokenizer, saved_model=saved_model)

In [20]:
# # ANLP SCINER TEST
# test_output_model(test_data_file="../data/anlp-sciner-test.txt", test_output_file= "../data/sciner-mysys1.conll", tokenizer=tokenizer, saved_model=saved_model)

In [21]:
# test_output_model(test_data_file="../bert.txt", test_output_file= "../bert-mysys1.conll", tokenizer=tokenizer, saved_model=saved_model)

In [22]:
# from transformers import pipeline, AutoModelForTokenClassification
# # Named entity recognition pipeline, passing in a specific model and tokenizer
# SCINER = pipeline('ner', model=saved_ckpt, tokenizer=tokenizer, aggregation_strategy='simple')

In [23]:
# test_data_file = "./anlp-sciner-test.txt"

# with open(test_data_file, 'rt', encoding='utf-8') as f:
#     paragraphs = f.read().split("\n")
# output = SCINER(paragraphs)

In [24]:
# test_output = "./test_ouput.conll"
# test_data_file = "./anlp-sciner-test.txt"

# with open(test_output, 'wt', encoding='utf-8') as f:
#     with open(test_data_file, 'rt', encoding='utf-8') as f:
#         paragraphs = f.read().split("\n")
#         for idx, pa in enumerate(paragraphs):
#             print(output[idx])
#             encoding = tokenizer.encode(pa)
#             # toks = pa.split(" ")
#             # ners = ['O'] * len(toks)
            
            
#             # for i,entities in enumerate(output[idx]):
#             #     for j, word in enumerate(toks):
#             #         st = entities['start']
#             #         ed = entities['end']
#             #         if ed != st:
#             #             f
#             # print(":)")
#             # # for i, tok in enumerate(pa.split(" ")):
#             # #     print(tok)


In [25]:
# def postprocess(predictions, labels):
#     predictions = predictions.detach().cpu().clone().numpy()
#     labels = labels.detach().cpu().clone().numpy()

#     # Remove ignored index (special tokens) and convert to labels
#     true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
#     true_predictions = [
#         [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
#         for prediction, label in zip(predictions, labels)
#     ]
#     return true_labels, true_predictions

In [26]:
# from torch.utils.data import DataLoader

# train_dataloader = DataLoader(
#     tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
# )
# eval_dataloader = DataLoader(
#     tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
# )